In [1]:
using Pkg
using JuMP
using Clp
using Cbc

In [11]:
#Model
m = Model(Cbc.Optimizer) 

#Variables for each position(matrix)
#X_ijk - 1 if the position i,j has value k
@variable(m, x[1:9, 1:9, 1:9], binary = true)

#Constraints of fixed positions
fixed_list = [1 3 5; 1 9 3; 2 5 4; 2 6 6;
3 3 7; 3 9 2; 4 2 1; 4 6 3; 
4 8 6; 4 9 9; 5 2 4; 5 4 6;
5 6 9; 5 8 5; 6 1 9; 6 2 8;
6 4 2; 6 8 7; 7 1 2; 7 7 9;
8 4 8; 8 5 1; 9 1 6; 9 7 4]
for i in 1:24
    row = fixed_list[i, 1]
    col = fixed_list[i, 2]
    value = fixed_list[i, 3]
    #Every other value is 0 than the specific value
    for j in 1:9
        if j == value
            @constraint(m, x[row, col, j] == 1)
        else
            @constraint(m, x[row, col, j] == 0)
        end
    end
end
for i in 1:9
    for j in 1:9
        #For each position  there is only one x_ijk != 0
        @constraint(m, sum(x[i, j, 1:9]) == 1)
        
        #For each line eand each value there is only one x_ijk != 0
        @constraint(m, sum(x[i, 1:9, j]) == 1)
        
        #For each column and each value there is only one x_ijk !=0
        @constraint(m, sum(x[1:9, i, j]) == 1)
    end
end

#For each 3x3 block and for each value there is only one x_ijk != 0
for i in [1, 4, 7]
    for j in [1, 4, 7]
        for k in 1:9
            @constraint(m, sum(x[i:i+2, j:j+2,k]) == 1)
        end
    end
end

optimize!(m)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 0 - 0.00 seconds
Cgl0002I 192 variables fixed
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 0 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
MixedIntegerRounding2 was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
FlowCover w

In [12]:
println("Final Sudoku")
result = JuMP.value.(x)
result_text = ""
for i in 1:9
    result_text = result_text * "\n"
    for j in 1:9
        for k in 1:9
            if result[i, j, k] == 1
                result_text = result_text * " " * string(k) 
            end
        end
    end
end
println(result_text)

Final Sudoku

 4 6 5 7 2 8 1 9 3
 1 2 9 3 4 6 7 8 5
 8 3 7 1 9 5 6 4 2
 5 1 2 4 7 3 8 6 9
 7 4 3 6 8 9 2 5 1
 9 8 6 2 5 1 3 7 4
 2 7 1 5 6 4 9 3 8
 3 9 4 8 1 7 5 2 6
 6 5 8 9 3 2 4 1 7


In [15]:
#New model adapted for simplex
#without using fixed binary variables and without using equality constraints

#Model
m_hat = Model(Cbc.Optimizer) 

#Variables for each position(matrix)
#X_ijk - 1 if the position i,j has value k
@variable(m_hat, x_hat[1:9, 1:9, 1:9], integer = true)
#Variable for find initial solution
@variable(m_hat, x0)

#Constraint for binary
for i in 1:9
    for j in 1:9
        for k in 1:9
            @constraint(m_hat, x_hat[i, j, k] <= 1)
            @constraint(m_hat, x_hat[i, j, k] >= 0)
        end
    end
end

#Constraint for x0 positive
@constraint(m_hat, x0 >= 0)

#Constraints of fixed positions
fixed_list = [1 3 5; 1 9 3; 2 5 4; 2 6 6;
3 3 7; 3 9 2; 4 2 1; 4 6 3; 
4 8 6; 4 9 9; 5 2 4; 5 4 6;
5 6 9; 5 8 5; 6 1 9; 6 2 8;
6 4 2; 6 8 7; 7 1 2; 7 7 9;
8 4 8; 8 5 1; 9 1 6; 9 7 4]
for i in 1:24
    row = fixed_list[i, 1]
    col = fixed_list[i, 2]
    value = fixed_list[i, 3]
    #Every other value is 0 than the specific value
    for j in 1:9
        if j == value
            @constraint(m_hat, x_hat[row, col, j] <= 1)
            @constraint(m_hat, x_hat[row, col, j] >= 1)
        else
            @constraint(m_hat, x_hat[row, col, j] <= 0)
            @constraint(m_hat, x_hat[row, col, j] >= 0)
        end
    end
end

for i in 1:9
    for j in 1:9
        #For each position  there is only one x_ijk != 0
        @constraint(m_hat, sum(x_hat[i, j, 1:9]) - x0 <= 1)
        @constraint(m_hat, - sum(x_hat[i, j, 1:9]) - x0 <= - 1)
        
        #For each line eand each value there is only one x_ijk != 0
        @constraint(m_hat, sum(x_hat[i, 1:9, j]) -x0 <= 1)
        @constraint(m_hat, - sum(x_hat[i, 1:9, j]) -x0 <= - 1)
        
        #For each column and each value there is only one x_ijk !=0
        @constraint(m_hat, sum(x_hat[1:9, i, j]) -x0 <= 1)
        @constraint(m_hat, - sum(x_hat[1:9, i, j]) -x0 <= -1)
    end
end

#For each 3x3 block and for each value there is only one x_ijk != 0
for i in [1, 4, 7]
    for j in [1, 4, 7]
        for k in 1:9
            @constraint(m_hat, sum(x_hat[i:i+2, j:j+2,k]) -x0 <= 1)
            @constraint(m_hat, - sum(x_hat[i:i+2, j:j+2,k]) -x0 <= - 1)
        end
    end
end

@objective(m_hat, Min, x0)
optimize!(m_hat)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 0 - 0.01 seconds
Cgl0004I processed model has 528 rows, 514 columns (513 integer (513 of which binary)) and 4194 elements
Cbc0012I Integer solution of 0 found by DiveCoefficient after 0 iterations and 0 nodes (0.02 seconds)
Cbc0001I Search completed - best objective 0, took 0 iterations and 0 nodes (0.02 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from 0 to 0
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and created 0 cuts of which 0 were active afte

In [17]:
println("Final Sudoku")
println("x0 value: ", JuMP.value.(x0))
result = JuMP.value.(x_hat)
result_text = ""
for i in 1:9
    result_text = result_text * "\n"
    for j in 1:9
        for k in 1:9
            if result[i, j, k] == 1
                result_text = result_text * " " * string(k) 
            end
        end
    end
end
println(result_text)

Final Sudoku
x0 value: 0.0

 4 6 5 7 2 8 1 9 3
 1 2 9 3 4 6 7 8 5
 8 3 7 1 9 5 6 4 2
 5 1 2 4 7 3 8 6 9
 7 4 3 6 8 9 2 5 1
 9 8 6 2 5 1 3 7 4
 2 7 1 5 6 4 9 3 8
 3 9 4 8 1 7 5 2 6
 6 5 8 9 3 2 4 1 7
